In [1]:
library(ArchR)
library(org.Ss.eg.db)
library(SuscrofaTxdb.11.108.fixed)
library(GenomicRanges)
library(gprofiler2)
library(GenomicFeatures)


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--' ,----`-,__ ___/'  --,-`-===================##========>
       \               '        ##_______ _____ ,--,__,=##,__   ///
        ,    __==    ___,-,__,--'#'  ==='      `-'    | ##,-/
        -,____,---'       \\####\\________________,--\\_##,/
           ___      .______        ______  __    __  .______      
          /   \     |   _ 

In [2]:
orgdb <- org.Ss.eg.db

In [3]:
txdb_double <- makeTxDbFromGFF("/home/adufour/work/genome_and_annotation/plus4pigs.mapped_to_reference.2023_01_18.Sus_scrofa.Sscrofa11.1.108.sorted.gtf", format="gtf")

Import genomic features from the file as a GRanges object ... 
OK

Prepare the 'metadata' data frame ... 
OK

Make the TxDb object ... 
Warning message in .get_cds_IDX(mcols0$type, mcols0$phase):
“The "phase" metadata column contains non-NA values for features of type
  stop_codon. This information was ignored.”
OK



In [4]:
txdb_double

TxDb object:
# Db type: TxDb
# Supporting package: GenomicFeatures
# Data source: /home/adufour/work/genome_and_annotation/plus4pigs.mapped_to_reference.2023_01_18.Sus_scrofa.Sscrofa11.1.108.sorted.gtf
# Organism: NA
# Taxonomy ID: NA
# miRBase build ID: NA
# Genome: NA
# Nb of transcripts: 60273
# Db created by: GenomicFeatures package from Bioconductor
# Creation time: 2023-07-28 15:31:23 +0200 (Fri, 28 Jul 2023)
# GenomicFeatures version at creation time: 1.46.1
# RSQLite version at creation time: 2.2.20
# DBSCHEMAVERSION: 1.2

In [5]:
genes <- GenomicFeatures::genes(txdb_double)

In [6]:
dataframe <- gconvert(
genes$gene_id,
organism = "sscrofa",
target = "ENSG",
mthreshold = 1,
filter_na = FALSE
)

In [7]:
genes$gene_id <- dataframe$target

In [8]:
genes$symbol <- dataframe$name

In [9]:
names(genes) <- NULL

In [10]:
genes <- genes[!genes$gene_id %in% NA]

In [11]:
filter <- seqlevels(genes)[!seqlevels(genes) %in% seqlevelsInUse(genes)]

In [12]:
seqlevels(genes) <- seqlevelsInUse(genes)

In [13]:
exons <- exons(txdb_double, columns="gene_id")

In [14]:
dataframe <- gconvert(
exons$gene_id,
organism = "sscrofa",
target = "ENSG",
mthreshold = 1,
filter_na = FALSE
)

In [15]:
exons$gene_id <- dataframe$target

In [16]:
exons$symbol <- dataframe$name

In [17]:
exons <- exons[!exons$gene_id %in% NA]

In [18]:
seqlevels(exons) <- seqlevelsInUse(exons)

In [19]:
transcripts_list <- transcripts(txdb_double, columns=c("tx_id", "tx_name"))

In [20]:
transcripts_list <- transcripts_list[seqnames(transcripts_list) %in% seqlevels(transcripts_list)[seqlevels(transcripts_list) %in% seqlevels(genes)]]

In [21]:
seqlevels(transcripts_list) <- seqlevelsInUse(transcripts_list)

In [22]:
tss <- resize(transcripts_list, width=1, fix='start')

In [23]:
geneAnnotation <- createGeneAnnotation(genes = genes, exons = exons, TSS = tss, OrgDb = orgdb)

In [24]:
geneAnnotation_old <- readRDS("/home/adufour/work/rds_storage/omics/geneannotation.rds")

In [25]:
saveRDS(geneAnnotation, file = "/home/adufour/work/rds_storage/omics/geneannotation.rds")

In [30]:
seqlevels(genes)

[1] "1"              "2"              "3"              "4"             
  [5] "5"              "6"              "7"              "8"             
  [9] "9"              "10"             "11"             "12"            
 [13] "13"             "14"             "15"             "16"            
 [17] "17"             "18"             "X"              "Y"             
 [21] "MT"             "AEMK02000127.1" "AEMK02000130.1" "AEMK02000131.1"
 [25] "AEMK02000133.1" "AEMK02000137.1" "AEMK02000139.1" "AEMK02000140.1"
 [29] "AEMK02000146.1" "AEMK02000147.1" "AEMK02000150.1" "AEMK02000151.1"
 [33] "AEMK02000153.1" "AEMK02000155.1" "AEMK02000156.1" "AEMK02000159.1"
 [37] "AEMK02000161.1" "AEMK02000162.1" "AEMK02000164.1" "AEMK02000170.1"
 [41] "AEMK02000171.1" "AEMK02000172.1" "AEMK02000173.1" "AEMK02000174.1"
 [45] "AEMK02000175.1" "AEMK02000182.1" "AEMK02000183.1" "AEMK02000186.1"
 [49] "AEMK02000188.1" "AEMK02000189.1" "AEMK02000191.1" "AEMK02000194.1"
 [53] "AEMK02000197.1" "AEMK02000200.1" "AEMK02000203.1" "AEMK02000204.1"
 [57] "AEMK02000207.1" "AEMK02000208.1" "AEMK02000213.1" "AEMK02000214.1"
 [61] "AEMK02000216.1" "AEMK02000222.1" "AEMK02000223.1" "AEMK02000224.1"
 [65] "AEMK02000227.1" "AEMK02000229.1" "AEMK02000234.1" "AEMK02000237.1"
 [69] "AEMK02000238.1" "AEMK02000240.1" "AEMK02000244.1" "AEMK02000245.1"
 [73] "AEMK02000247.1" "AEMK02000248.1" "AEMK02000253.1" "AEMK02000254.1"
 [77] "AEMK02000258.1" "AEMK02000259.1" "AEMK02000260.1" "AEMK02000261.1"
 [81] "AEMK02000263.1" "AEMK02000264.1" "AEMK02000265.1" "AEMK02000273.1"
 [85] "AEMK02000276.1" "AEMK02000277.1" "AEMK02000278.1" "AEMK02000279.1"
 [89] "AEMK02000280.1" "AEMK02000282.1" "AEMK02000287.1" "AEMK02000289.1"
 [93] "AEMK02000291.1" "AEMK02000295.1" "AEMK02000297.1" "AEMK02000299.1"
 [97] "AEMK02000304.1" "AEMK02000308.1" "AEMK02000310.1" "AEMK02000312.1"
[101] "AEMK02000313.1" "AEMK02000316.1" "AEMK02000319.1" "AEMK02000320.1"
[105] "AEMK02000324.1" "AEMK02000326.1" "AEMK02000327.1" "AEMK02000328.1"
[109] "AEMK02000329.1" "AEMK02000333.1" "AEMK02000336.1" "AEMK02000340.1"
[113] "AEMK02000341.1" "AEMK02000346.1" "AEMK02000348.1" "AEMK02000351.1"
[117] "AEMK02000355.1" "AEMK02000356.1" "AEMK02000361.1" "AEMK02000363.1"
[121] "AEMK02000364.1" "AEMK02000368.1" "AEMK02000369.1" "AEMK02000370.1"
[125] "AEMK02000374.1" "AEMK02000375.1" "AEMK02000378.1" "AEMK02000379.1"
[129] "AEMK02000380.1" "AEMK02000383.1" "AEMK02000388.1" "AEMK02000389.1"
[133] "AEMK02000390.1" "AEMK02000391.1" "AEMK02000393.1" "AEMK02000394.1"
[137] "AEMK02000399.1" "AEMK02000400.1" "AEMK02000402.1" "AEMK02000403.1"
[141] "AEMK02000404.1" "AEMK02000408.1" "AEMK02000410.1" "AEMK02000412.1"
[145] "AEMK02000414.1" "AEMK02000415.1" "AEMK02000418.1" "AEMK02000421.1"
[149] "AEMK02000422.1" "AEMK02000423.1" "AEMK02000425.1" "AEMK02000426.1"
[153] "AEMK02000427.1" "AEMK02000431.1" "AEMK02000434.1" "AEMK02000435.1"
[157] "AEMK02000437.1" "AEMK02000438.1" "AEMK02000439.1" "AEMK02000440.1"
[161] "AEMK02000442.1" "AEMK02000446.1" "AEMK02000448.1" "AEMK02000449.1"
[165] "AEMK02000451.1" "AEMK02000452.1" "AEMK02000453.1" "AEMK02000457.1"
[169] "AEMK02000459.1" "AEMK02000460.1" "AEMK02000465.1" "AEMK02000467.1"
[173] "AEMK02000468.1" "AEMK02000472.1" "AEMK02000473.1" "AEMK02000474.1"
[177] "AEMK02000476.1" "AEMK02000477.1" "AEMK02000478.1" "AEMK02000479.1"
[181] "AEMK02000482.1" "AEMK02000484.1" "AEMK02000485.1" "AEMK02000489.1"
[185] "AEMK02000490.1" "AEMK02000491.1" "AEMK02000493.1" "AEMK02000495.1"
[189] "AEMK02000496.1" "AEMK02000497.1" "AEMK02000499.1" "AEMK02000500.1"
[193] "AEMK02000501.1" "AEMK02000503.1" "AEMK02000506.1" "AEMK02000507.1"
[197] "AEMK02000509.1" "AEMK02000510.1" "AEMK02000511.1" "AEMK02000514.1"
[201] "AEMK02000515.1" "AEMK02000517.1" "AEMK02000520.1" "AEMK02000522.1"
[205] "AEMK02000525.1" "AEMK02000526.1" "AEMK02000527.1" "AEMK02000528.1"
[209] "AEMK02000529.1" "AEMK02000531.1" "AEMK02000532.1" "AEMK02000534.1"
[213] "AEMK02000537.1" "AEMK02000541.1" "AEMK02000544.1" "AEMK02000546.1"
[217] 